## *If found useful, please leave an Upvote* 👍

In [ ]:
import os
from IPython.display import Image
Image(filename="../input/front-image/Statisical-cash-forecasting-methods-1.png", width= 1200, height=800)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime, timedelta
from tqdm import tqdm 
 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### Install PyCaret

In [ ]:
# PyCaret might not be installed. Below commands would install PyCaret and import necessary regression models
!pip install pycaret
from pycaret.regression import *

In [ ]:
# Read Data
data = pd.read_csv('/kaggle/input/retail-analysis-with-walmart-data/Walmart_Store_sales.csv')

In [ ]:
# Data Properties
data.dtypes

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
# Converting all columns to Upper Case 
data.columns = [_c.upper() for _c in data.columns]

In [ ]:
# Converting datatype of Date to DateTime and creating a month column
data['DATE'] = pd.to_datetime(data['DATE'])
data.loc[:,'MONTH'] = data['DATE'].dt.month


In [ ]:
data.head()

#### Creating train and test data dictionaries

In [ ]:
train_dict = {}
test_dict = {}

unique_stores = list(data['STORE'].unique())

for i in unique_stores:
    train_dict[i] = data.loc[data['STORE'] == i].copy()
    train_dict[i].sort_values(by = 'DATE', inplace = True)
    train_dict[i].reset_index(inplace = True, drop = True)
    test_dict[i] = train_dict[i].iloc[-6:,:].copy()
    train_dict[i] = train_dict[i].iloc[0:len(train_dict[i])-6,:]
    test_dict[i].reset_index(inplace = True, drop = True)

#### Setup PyCaret and Run Multiple Regression Models for each Store

In [ ]:
# Executing PyCaret and Saving the output
output_dict = {}
all_results = []
final_model = {}
for i in tqdm(unique_stores):
    
    df_subset = train_dict[i]
    
    # initialize setup from pycaret.regression
    s = setup(df_subset, target = 'WEEKLY_SALES', train_size = 0.95,
              data_split_shuffle = False, fold_strategy = 'timeseries', fold = 3,
              ignore_features = ['DATE', 'STORE'],
              numeric_features = ['TEMPERATURE', 'FUEL_PRICE', 'CPI', 'UNEMPLOYMENT'],
              categorical_features = ['MONTH', 'HOLIDAY_FLAG'],
              silent = True, verbose = False, session_id = 123)
    
    # compare all models and select best one based on MAE
    best_model = compare_models(sort = 'MAE', verbose=False)
    
    # capture the compare result grid and store best model in list
    p = pull().iloc[0:1]
    p['time_series'] = str(i)
    all_results.append(p)
    
    # finalize model i.e. fit on entire data including test set
    f = finalize_model(best_model)
    
    # attach final model to a dictionary
    final_model[i] = f
    
    # Predict
    score_df = test_dict[i].drop('WEEKLY_SALES', axis = 1)
    p = predict_model(f, data=score_df)
    test_dict[i].loc[:,'FORECASTED_SALES'] = p['Label']

#### Check Different best performing models for each store

In [ ]:
# Checking Models used for Each Store 
concat_results = pd.concat(all_results,axis=0)
concat_results

#### Plotting

In [ ]:
# Plotting Result of any Store 17 randomly
import plotly.express as px
df = test_dict[17][['DATE', 'WEEKLY_SALES', 'FORECASTED_SALES']]
fig = px.line(df, x="DATE", y=df.columns,
              title='Weekly Sales')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()

#### This could be used as a baseline model to further check for trend, seasonality in the data and account for it